In [82]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix

import plotly.express as px

import xgboost as xgb

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [83]:
prediction_season = 2023

In [84]:
# Base data frames

point_spread_base = pd.read_csv('../pipeline_season_testing/data/final_predictive_dataframe.csv').dropna()
over_under_base = pd.read_csv('../pipeline_season_testing/data/final_predictive_dataframe.csv').dropna()
game_outcome_base = pd.read_csv('../pipeline_season_testing/data/final_predictive_dataframe.csv').dropna()
gambling_base = pd.read_csv('../pipeline_season_testing/data/final_predictive_dataframe.csv').dropna()

In [85]:
# Get relevant prediction season

point_spread_base = point_spread_base[point_spread_base.season.isin([prediction_season])]
over_under_base = over_under_base[over_under_base.season.isin([prediction_season])]
game_outcome_base = game_outcome_base[game_outcome_base.season.isin([prediction_season])]
gambling_base = gambling_base[gambling_base.season.isin([prediction_season])]

In [86]:
gambling_base

,Unnamed: 0,season,week,home,away,home_qb,away_qb,passing_value_adjusted_home,rushing_value_adjusted_home,pass_def_value_adjusted_home,rush_def_value_adjusted_home,special_teams_value_home,passing_value_adjusted_away,rushing_value_adjusted_away,pass_def_value_adjusted_away,rush_def_value_adjusted_away,special_teams_value_away,total_possession_time_standardized_home,total_possession_time_standardized_away,total_plays_standardized_home,total_plays_standardized_away,pass_percentage_standardized_home,pass_percentage_standardized_away,home_moneyline,away_moneyline,home_spread,away_spread,total_score_line,home_implied_prob,away_implied_prob
1,1,2023,1,PIT,SF,K. Pickett,B. Purdy,0.780551,0.181826,-0.187822,0.393845,-0.235606,1.126517,0.303352,0.454628,-0.314641,0.261315,0.491132,0.483027,0.580801,-0.005191,-0.893167,-0.768058,-105,-125,2.0,-2.0,41.5,0.512195,0.555556
4,4,2023,1,CLE,CIN,D. Watson,J. Burrow,-0.211152,0.442853,0.359798,0.329270,-0.172747,0.314650,0.345601,-0.234975,0.610355,0.175614,-0.218433,0.109044,-0.504155,0.308112,-0.335390,0.787408,102,-122,1.5,-1.5,47.0,0.495050,0.549550
8,8,2023,1,NE,PHI,M. Jones,J. Hurts,-0.240548,-0.266291,-0.275203,0.382505,-0.757802,0.512212,0.642981,-0.078336,-0.474858,0.262363,-0.331901,1.108068,-0.306890,0.853491,0.631381,-0.910959,154,-185,3.5,-3.5,44.5,0.393701,0.649123
9,9,2023,1,LAC,MIA,J. Herbert,T. Tagovailoa,0.620026,-0.263069,0.441181,-0.943062,0.466609,-0.338186,0.455923,0.346837,0.420816,0.200025,0.269405,-1.028343,0.435754,-1.496281,0.429984,0.506390,-162,136,-3.0,3.0,51.0,0.618321,0.423729
10,10,2023,1,SEA,LAR,G. Smith,M. Stafford,-0.407814,0.324747,-0.163599,-0.257049,-0.224850,0.054568,-0.629966,0.220240,-0.403564,-0.120553,0.299123,-0.013109,0.569197,-0.550570,-0.209923,-0.032649,-225,185,-5.0,5.0,46.0,0.692308,0.350877
13,13,2023,1,NYG,DAL,D. Jones,D. Prescott,-0.098699,1.036148,-0.237257,-1.111900,-0.441338,0.244841,-0.494522,0.018246,0.194036,-0.195399,-0.174821,-0.160734,-0.324296,0.389338,-0.014257,0.061580,140,-166,3.0,-3.0,45.0,0.416667,0.624060


In [87]:
# Feature lists

point_spread_feature_list = [
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home'
    , 'pass_def_value_adjusted_home'
    , 'rush_def_value_adjusted_home'
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away'
    , 'rushing_value_adjusted_away'
    , 'pass_def_value_adjusted_away'
    , 'rush_def_value_adjusted_away'
    , 'special_teams_value_away'
    , 'total_possession_time_standardized_home'
    , 'total_possession_time_standardized_away'
    , 'total_plays_standardized_home'
    , 'total_plays_standardized_away'
    , 'pass_percentage_standardized_home'
    , 'pass_percentage_standardized_away'
    , 'home_spread'
]

over_under_feature_list = [
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home'
    , 'pass_def_value_adjusted_home'
    , 'rush_def_value_adjusted_home'
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away'
    , 'rushing_value_adjusted_away'
    , 'pass_def_value_adjusted_away'
    , 'rush_def_value_adjusted_away'
    , 'special_teams_value_away'
    , 'total_possession_time_standardized_home'
    , 'total_possession_time_standardized_away'
    , 'total_plays_standardized_home'
    , 'total_plays_standardized_away'
    , 'pass_percentage_standardized_home'
    , 'pass_percentage_standardized_away'
    , 'total_score_line'
]

game_outcome_feature_list =[
    'passing_value_adjusted_home'
    , 'rushing_value_adjusted_home' 
    , 'pass_def_value_adjusted_home' 
    , 'rush_def_value_adjusted_home' 
    , 'special_teams_value_home'
    , 'passing_value_adjusted_away' 
    , 'rushing_value_adjusted_away' 
    , 'pass_def_value_adjusted_away' 
    , 'rush_def_value_adjusted_away' 
    , 'special_teams_value_away'
]

In [88]:
# Get features

point_spread_features = point_spread_base[point_spread_feature_list].to_numpy()
over_under_features = over_under_base[over_under_feature_list].to_numpy()
game_outcome_features = game_outcome_base[game_outcome_feature_list].to_numpy()

In [89]:
# Import models

point_spread_model = load('../modeling/point_spread_modeling/saved_models/ps_xg_boost_av.joblib')
over_under_model = load('../modeling/over_under_modeling/saved_models/ou_xg_boost_kinda_ok_av.joblib')
game_outcome_model = load('../modeling/game_outcome_modeling/saved_models/xg_boost_kinda_ok_av.joblib')

In [90]:
# Point Spread Predictions

home_cover_preds = point_spread_model.predict(point_spread_features)
home_cover_probs = point_spread_model.predict_proba(point_spread_features)

In [91]:
# Over Under Predictions

over_hits_preds = over_under_model.predict(over_under_features)
over_hits_probs = over_under_model.predict_proba(over_under_features)

In [92]:
# Game Winner Predictions

home_win_preds = game_outcome_model.predict(game_outcome_features)
home_win_probs = game_outcome_model.predict_proba(game_outcome_features)

In [93]:
point_spread_base['home_cover_pred'] = home_cover_preds
point_spread_base['home_cover_prob'] = home_cover_probs[:,1]

In [94]:
over_under_base['over_hits_pred'] = over_hits_preds
over_under_base['over_hits_prob'] = over_hits_probs[:,1]

In [95]:
game_outcome_base['home_win_pred'] = home_win_preds
game_outcome_base['home_win_prob'] = home_win_probs[:,1]

In [96]:
# Merge into one data frame

gambling_base = gambling_base[['season', 'week', 'home', 'away', 'home_qb', 'away_qb', 'home_spread', 'away_spread', 'total_score_line', 'home_implied_prob', 'away_implied_prob', 'home_moneyline', 'away_moneyline']]
gambling_base = gambling_base.merge(point_spread_base[['season', 'week', 'home', 'away', 'home_qb', 'away_qb', 'home_cover_pred', 'home_cover_prob']])
gambling_base = gambling_base.merge(over_under_base[['season', 'week', 'home', 'away', 'home_qb', 'away_qb', 'over_hits_pred', 'over_hits_prob']])
gambling_base = gambling_base.merge(game_outcome_base[['season', 'week', 'home', 'away', 'home_qb', 'away_qb', 'home_win_pred', 'home_win_prob']])

In [97]:
gambling_base['home_cover_advantage'] = gambling_base.apply(lambda x: x.home_cover_prob-.5, axis=1)
gambling_base['away_cover_advantage'] = gambling_base.apply(lambda x: (1-x.home_cover_prob)-.5, axis=1)

In [98]:
gambling_base['over_advantage'] = gambling_base.apply(lambda x: x.over_hits_prob-.5, axis=1)
gambling_base['under_advantage'] = gambling_base.apply(lambda x: (1-x.over_hits_prob)-.5, axis=1)

In [99]:
gambling_base['home_win_advantage'] = gambling_base.apply(lambda x: x.home_win_prob-x.home_implied_prob, axis=1)
gambling_base['away_win_advantage'] = gambling_base.apply(lambda x: (1-x.home_win_prob)-x.away_implied_prob, axis=1)

In [100]:
gambling_base

,season,week,home,away,home_qb,away_qb,home_spread,away_spread,total_score_line,home_implied_prob,away_implied_prob,home_moneyline,away_moneyline,home_cover_pred,home_cover_prob,over_hits_pred,over_hits_prob,home_win_pred,home_win_prob,home_cover_advantage,away_cover_advantage,over_advantage,under_advantage,home_win_advantage,away_win_advantage
0,2023,1,PIT,SF,K. Pickett,B. Purdy,2.0,-2.0,41.5,0.512195,0.555556,-105,-125,0,0.371744,1,0.737932,0,0.406733,-0.128256,0.128256,0.237932,-0.237932,-0.105462,0.037712
1,2023,1,CLE,CIN,D. Watson,J. Burrow,1.5,-1.5,47.0,0.495050,0.549550,102,-122,0,0.134089,1,0.616254,1,0.567349,-0.365911,0.365911,0.116254,-0.116254,0.072300,-0.116899
2,2023,1,NE,PHI,M. Jones,J. Hurts,3.5,-3.5,44.5,0.393701,0.649123,154,-185,0,0.045123,0,0.134856,0,0.165219,-0.454877,0.454877,-0.365144,0.365144,-0.228482,0.185658
3,2023,1,LAC,MIA,J. Herbert,T. Tagovailoa,-3.0,3.0,51.0,0.618321,0.423729,-162,136,0,0.073635,1,0.558629,1,0.562957,-0.426365,0.426365,0.058629,-0.058629,-0.055364,0.013314
4,2023,1,SEA,LAR,G. Smith,M. Stafford,-5.0,5.0,46.0,0.692308,0.350877,-225,185,0,0.114545,1,0.582316,0,0.123178,-0.385455,0.385455,0.082316,-0.082316,-0.569129,0.525945
5,2023,1,NYG,DAL,D. Jones,D. Prescott,3.0,-3.0,45.0,0.416667,0.624060,140,-166,0,0.254470,1,0.509505,1,0.602668,-0.245530,0.245530,0.009505,-0.009505,0.186001,-0.226728


In [101]:
def get_point_spread_bet(game, threshold):
    if game.home_cover_advantage > game.away_cover_advantage:
        bet_team = 'home'
        bet_advantage = game.home_cover_advantage
    else:
        bet_team = 'away'
        bet_advantage = game.away_cover_advantage
        
    if bet_advantage < threshold:
        return 'none'
    
    else:
        return bet_team
    
def get_over_under_bet(game, threshold):
    if game.over_advantage > game.under_advantage:
        bet_team = 'over'
        bet_advantage = game.over_advantage
    else:
        bet_team = 'under'
        bet_advantage = game.under_advantage
        
    if bet_advantage < threshold:
        return 'none'
    
    else:
        return bet_team
    
def get_game_outcome_bet(game, threshold):
    if game.home_win_advantage > game.away_win_advantage:
        bet_team = 'home'
        bet_advantage = game.home_win_advantage
    else:
        bet_team = 'away'
        bet_advantage = game.away_win_advantage
        
    if bet_advantage < threshold:
        return 'none'
    
    else:
        return bet_team

In [102]:
gambling_final = gambling_base.copy()
gambling_final['point_spread_algo_bet'] = gambling_final.apply(lambda x: get_point_spread_bet(x, .4), axis=1)
gambling_final['over_under_algo_bet'] = gambling_final.apply(lambda x: get_over_under_bet(x, .07), axis=1)
gambling_final['game_outcome_algo_bet'] = gambling_final.apply(lambda x: get_game_outcome_bet(x, .15), axis=1)

In [103]:
def get_print_bet(game, bet_type='spread'):
    if bet_type == 'spread' and game.point_spread_algo_bet != 'none':
        return '{}, {}'.format(game[game.point_spread_algo_bet], game['{}_spread'.format(game.point_spread_algo_bet)])
    if bet_type == 'over_under' and game.over_under_algo_bet != 'none':
        return '{}'.format(game.over_under_algo_bet.capitalize())
    if bet_type == 'outcome' and game.game_outcome_algo_bet != 'none':
        return '{}, {}'.format(game[game.game_outcome_algo_bet], game['{}_moneyline'.format(game.game_outcome_algo_bet)])
    else:
        return None

In [104]:
gambling_final

,season,week,home,away,home_qb,away_qb,home_spread,away_spread,total_score_line,home_implied_prob,away_implied_prob,home_moneyline,away_moneyline,home_cover_pred,home_cover_prob,over_hits_pred,over_hits_prob,home_win_pred,home_win_prob,home_cover_advantage,away_cover_advantage,over_advantage,under_advantage,home_win_advantage,away_win_advantage,point_spread_algo_bet,over_under_algo_bet,game_outcome_algo_bet
0,2023,1,PIT,SF,K. Pickett,B. Purdy,2.0,-2.0,41.5,0.512195,0.555556,-105,-125,0,0.371744,1,0.737932,0,0.406733,-0.128256,0.128256,0.237932,-0.237932,-0.105462,0.037712,none,over,none
1,2023,1,CLE,CIN,D. Watson,J. Burrow,1.5,-1.5,47.0,0.495050,0.549550,102,-122,0,0.134089,1,0.616254,1,0.567349,-0.365911,0.365911,0.116254,-0.116254,0.072300,-0.116899,none,over,none
2,2023,1,NE,PHI,M. Jones,J. Hurts,3.5,-3.5,44.5,0.393701,0.649123,154,-185,0,0.045123,0,0.134856,0,0.165219,-0.454877,0.454877,-0.365144,0.365144,-0.228482,0.185658,away,under,away
3,2023,1,LAC,MIA,J. Herbert,T. Tagovailoa,-3.0,3.0,51.0,0.618321,0.423729,-162,136,0,0.073635,1,0.558629,1,0.562957,-0.426365,0.426365,0.058629,-0.058629,-0.055364,0.013314,away,none,none
4,2023,1,SEA,LAR,G. Smith,M. Stafford,-5.0,5.0,46.0,0.692308,0.350877,-225,185,0,0.114545,1,0.582316,0,0.123178,-0.385455,0.385455,0.082316,-0.082316,-0.569129,0.525945,none,over,away
5,2023,1,NYG,DAL,D. Jones,D. Prescott,3.0,-3.0,45.0,0.416667,0.624060,140,-166,0,0.254470,1,0.509505,1,0.602668,-0.245530,0.245530,0.009505,-0.009505,0.186001,-0.226728,none,none,home


In [105]:
check_week = 1
gambling_final_week_check = gambling_final[gambling_final.week == check_week]

for index, game in gambling_final_week_check.iterrows():
    print('{} @ {}'.format(game.away, game.home))
    print()
    if get_print_bet(game, 'spread'):
        print(get_print_bet(game, 'spread'))
    if get_print_bet(game, 'outcome'):
        print(get_print_bet(game, 'outcome'))
    if get_print_bet(game, 'over_under'):
        print(get_print_bet(game, 'over_under'))
    print()

SF @ PIT

Over

CIN @ CLE

Over

PHI @ NE

PHI, -3.5
PHI, -185
Under

MIA @ LAC

MIA, 3.0

LAR @ SEA

LAR, 185
Over

DAL @ NYG

NYG, 140

